In [1]:
import numpy as np
import matplotlib.pyplot as plt
from IPython.display import HTML
from matplotlib.animation import FuncAnimation
from math import sqrt
from scipy.special import erfc
from tqdm import tqdm_notebook

In [2]:
def getPrecise(t):
    res = np.zeros(N)
    res[0] = T0
    if t > 0:
        for i in range(N):
            res[i] = T0*erfc(i*dx/(2*sqrt(L*t*dt)))
        
    return res

In [3]:
def getPrecise_nelin(t):
    res = np.zeros(N)
#     res[0] = getT(dt*t)
    for i in range(N):
        if i*dx <= c*t*dt:
            res[i] = pow(sigma*c/L0*(c*t*dt-i*dx), 1/sigma)
            
    return res

# Линейное уравнение теплопроводности

$$
\frac{\partial{T}}{\partial{t}} = \lambda \frac{\partial^2{T}}{\partial{x^2}},  \lambda > 0
$$

In [14]:
L = 1 # lambda
T0 = 10

dt = 0.1
dx = 0.45

N = 200
T = 6000

sigma = L*dt/dx**2
print("Число Куранта = ", sigma)

assert sigma <= 0.5, "Схема неустойчива!"

Число Куранта =  0.49382716049382713


In [26]:
%%time
A = np.zeros((T,N))

for n in range(-1, T-1):
    A[n+1][0] = T0
    for i in range(1, N-1):
        A[n+1][i] = A[n][i] + sigma*(A[n][i+1]-2*A[n][i]+A[n][i-1])
    A[n+1][N-1] = 0

Wall time: 2.21 s


In [16]:
from numba import jit

In [42]:
%%time
A = np.zeros((T,N))

@jit
def new_iteration(A, n):
    tmp = np.zeros(N)
    tmp[0] = T0
    for i in range(1, N-1):
        tmp[i] = A[i] + sigma*(A[i+1]-2*A[i]+A[i-1])
    tmp[N-1] = 0
    
    return tmp


for n in range(-1, T-1):
    A[n+1] = new_iteration(A[n], n)

Wall time: 174 ms


In [29]:
fig, ax = plt.subplots(figsize = (12, 7))
xdata, ydata = range(N), []
ax.grid(True)
ln, = plt.plot([], [], color = 'g', linestyle = '-')
ln2, = plt.plot([], [], color = 'c', linestyle = ':')

def init():
    ax.set_xlim(0, N-1)
    ax.set_ylim(0, T0*1.1)
    return ln,

def update(t):
    ydata = A[frame_const*t]
    ln.set_data(xdata, ydata)
    ln2.set_data(xdata, getPrecise(frame_const*t))
    return ln,

frame_const = int(T*dt/2)
frame_const = 15
anim = FuncAnimation(fig, update, frames=tqdm_notebook(range(T//frame_const)), init_func=init, interval=50, blit=True)
html5video = anim.to_html5_video()
plt.close()
HTML(html5video)